# Network feature engineering - node/edge stats

This notebook starts from the file generated at step 01 (notebook '01_instrumentsFeatures.ipynb') and performs feature engineering adding useful node/edge stats

## Data import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
import itertools

In [4]:
#Instruments dataset
#import data
user = os.environ["USERNAME"]

#from home
filename = "190108_instrumentsdf2.pkl"
datafolder = "C:/Users/{}/Dropbox/University/MscDataScience_Birkbeck/thesis_project/data/".format(user)

#from work
#filename = "181109_instruments_merged.pkl"
#datafolder = "C:/Users/{}/Tradeteq Dropbox/Tradeteq Team/Clients/#GoFactoring/data analysis/".format(user)

inst = pd.read_pickle(datafolder+filename)
inst.head()

,customer_id,customer_name_1,debtor_id,debtor_name_1,invoice_number,invoice_date,due_date,invoice_amount,purchase_amount,purchase_amount_open,...,total_repayment,total_impairment,is_open,we_payment_share,has_purchase,has_deduction,is_due,has_discharge,cash_check,unexpl
uid,,,,,,,,,,,,,,,,,,,,,
2744:79/231,2004008,jobs united GmbH,79,Quadroni Linard,2744,2013-07-23,2013-08-02,913.70,0.0,0.0,...,0.00,0.00,False,NaN,False,False,True,True,913.70,True
2861:79/232,2004008,jobs united GmbH,79,Quadroni Linard,2861,2013-07-30,2013-08-09,2233.45,0.0,0.0,...,0.00,0.00,False,NaN,False,False,True,True,2233.45,True
2932:79/233,2004008,jobs united GmbH,79,Quadroni Linard,2932,2013-08-06,2013-08-16,1370.50,0.0,0.0,...,1370.50,0.00,False,1.0,False,False,True,False,0.00,False
1472:489/688,2004009,PM Personal GmbH,489,Style Interiors,1472,2013-08-13,2013-08-23,9195.10,0.0,0.0,...,0.00,0.00,False,NaN,False,False,True,True,9195.10,True
2042:512/645,2004009,PM Personal GmbH,512,Elektropartner AG,2042,2013-08-13,2013-08-23,4594.60,0.0,0.0,...,164.35,164.35,False,1.0,False,False,True,True,4265.90,True


In [5]:
print("{:} instruments".format(inst.shape[0]))

59820 instruments


In [6]:
print("{:} open, {:} with no payments".format(sum(inst.document_status=="offen") ,sum(inst.last_payment_date.isnull())))

8213 open, 12181 with no payments


### 1. Retrieving first and last posting date and payment date mismatch

In [7]:
inst["first_posting_date"] = inst.posting_date.apply(lambda x: min(x))
inst["last_posting_date"] = inst.posting_date.apply(lambda x: max(x))
inst["payment_date_mismatch"] = (inst.last_payment_date - inst.due_date).dt.days

### 2. Isolating date columns and calculating offsets from invoice_date

In [8]:
datecol = [  'due_date', 
           #"invoice_date", #almost always the earliest
       'discharge_date',  'input_date', 'creation_date', 
       'debt_collection_date', 'last_payment_date', 'reminder_date',
       'cancellation_date', 'value_date',
       'first_posting_date', 'last_posting_date']

for c in datecol:
    inst["dd_"+c] = (inst[c] - inst.invoice_date).apply(lambda x: x.days)

In [10]:
inst[['dd_'+i for i in datecol]].head()

,dd_due_date,dd_discharge_date,dd_input_date,dd_creation_date,dd_debt_collection_date,dd_last_payment_date,dd_reminder_date,dd_cancellation_date,dd_value_date,dd_first_posting_date,dd_last_posting_date
uid,,,,,,,,,,,
2744:79/231,10,973.0,921,921,NaN,NaN,48.0,NaN,10,921,973
2861:79/232,10,966.0,914,914,NaN,NaN,55.0,NaN,10,914,966
2932:79/233,10,NaN,907,907,NaN,907.0,48.0,NaN,10,907,959
1472:489/688,10,952.0,900,900,NaN,NaN,69.0,NaN,10,900,952
2042:512/645,10,952.0,900,900,NaN,900.0,125.0,NaN,10,900,952


### 3. Marking up instruments with buyer/seller relationship details properties known at instrument inception

For each instrument, stats will be added and classified depending on their nature.  
The type of stats can be of 3 types, and it will be named accordingly:  
- edge stats, referred to a customer/debtor pair - it will have the prefix 'cd'
- node stats referred to a customer - it will have the prefix 'c'
- node stats referred to a debtor - it will have the prefix 'd'

In [11]:
#UTILS

from scipy import stats

def series_trend(s, applylog=True):
    """
    This function defines a trend for a particular given series using linear regression.
    To be used with invoice_amount for the current dataset, in order to establish the entity of the transactions.
    """
    x=np.arange(s.shape[0])
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,s)
    #print(slope)
    if applylog:
        res = 0 if np.abs(slope)<1e-8 else np.sign(slope) * np.log(np.abs(slope))
    else:
        res = slope
    return res

decision_date_col = "value_date" #this is the main reference date for each instrument to establish if it has been repaid on time or if it is pastdue

def add_node_stats(inst, igroup, idx, id, ii, prefix):
    """
    This function adds stats to each node.
    inst: instruments dataframe sorted by invoice_date
    igroup: group of instruments between a certain buyer and a certain seller
    idx: instrument index in the igroup 
    id: instrument id
    ii: instrument features (literally the dataset sliced in correspondence of that instrument)
    """
    #adding counter of previously lent in this customer/debtor pair (inst is sorted by invoice date)
    inst.loc[id, prefix+"lent_c"] = idx 
        
    #adding counter of previously repaid instruments in this customer/debtor pair
    #to be repaid, the last payment date needs to be smaller than all the instrument date and the instrument needs to not be open
    repaid = (igroup.loc[:, "last_payment_date"] < ii[decision_date_col]) & (~ igroup.loc[:, "is_open"]) #filter for repaid instruments in this customer/debtor pair
    inst.loc[id, prefix+"repaid_c"] = sum(repaid) 
            
    #adding counter of previously impaired in this customer/debtor pair
    inst.loc[id, prefix+"impaired1_c"] = sum(igroup.loc[repaid,"has_impairment1"])
    inst.loc[id, prefix+"impaired2_c"] = sum(igroup.loc[repaid,"has_impairment2"])
        
    #counter of overdue in this customer/debtor pair (considering previous instruments)
    previous = igroup.index[:idx] #previous instruments selector
    inst.loc[id, prefix+"pastdue90_c"] = sum((igroup.loc[previous,"due_date"] < ii[decision_date_col] - datetime.timedelta(90)) & igroup.loc[previous,"is_pastdue90"])
    inst.loc[id, prefix+"pastdue180_c"] = sum((igroup.loc[previous,"due_date"] < ii[decision_date_col] - datetime.timedelta(180)) & igroup.loc[previous,"is_pastdue180"])
        
    #adding trend in amount lent in this customer/debtor pair
    inst.loc[id, prefix+"trend_a"] = 0 if idx<2 else series_trend(igroup.loc[previous,"invoice_amount"])
        
    #adding counter of weekend payments in this pair
    inst.loc[id, prefix+"we_payment_share"] = igroup.loc[repaid, "we_payment_share"].agg("mean")
        
    #adding payment_date_mismatch stats
    inst.loc[id, prefix+"pd_mismatch_mean"] = igroup.loc[repaid, "payment_date_mismatch"].agg("mean")
    inst.loc[id, prefix+"pd_mismatch_std"] = igroup.loc[repaid, "payment_date_mismatch"].agg("std") 

### 3.1 Adding buyer/seller pair attributes (cd)

In [12]:
#Adding buyer/seller pair attributes
print("Adding buyer/seller pair attributes...")
prefix = "cd_" #stands for customer/debtor
g_cb = inst.groupby(["customer_name_1", "debtor_name_1"])
for (customer, debtor), igroup in g_cb:
    #for each instrument in this group, already sorted by invoice_date
    for idx, (id, ii) in enumerate(igroup.iterrows()):
       add_node_stats(inst, igroup, idx, id, ii, prefix)

#Adding the ratio columns for the previously calculated stats
cl = [prefix+"repaid_", prefix+"impaired1_", prefix+"impaired2_", 
      prefix+"pastdue90_", prefix+"pastdue180_"]
for c in cl:
    inst[c+"r"] = inst[c+"c"] / inst[prefix+"lent_c"]

Adding buyer/seller pair attributes...


### 3.2 Adding buyer attributes (d)

In [13]:
print("Adding buyer attributes...")
#buyer attributes
prefix = "d_"
g_b = inst.groupby(["debtor_name_1"])
for _, igroup in g_b:
    #for each instrument in this group, already sorted by invoice_date
    for idx, (id, ii) in enumerate(igroup.iterrows()):
        add_node_stats(inst, igroup, idx, id, ii, prefix)      
#add the ratio columns
cl = [prefix+"repaid_", prefix+"impaired1_", prefix+"impaired2_", 
      prefix+"pastdue90_", prefix+"pastdue180_"]
for c in cl:
    inst[c+"r"] = inst[c+"c"] / inst[prefix+"lent_c"]

Adding buyer attributes...


### 3.3 Adding seller attributes (c)

In [14]:
#Adding seller attributes
print("Adding seller attributes...")
prefix = "c_"
g_b = inst.groupby(["customer_name_1"])
for _, igroup in g_b:
    #for each instrument in this group, already sorted by invoice_date
    for idx, (id, ii) in enumerate(igroup.iterrows()):
        add_node_stats(inst, igroup, idx, id, ii, prefix)      
#add the ratio columns
cl = [prefix+"repaid_", prefix+"impaired1_", prefix+"impaired2_", 
      prefix+"pastdue90_", prefix+"pastdue180_"]
for c in cl:
    inst[c+"r"] = inst[c+"c"] / inst[prefix+"lent_c"]

Adding seller attributes...


In [15]:
outputfile = 'instruments_with_nstats.pkl'
inst.to_pickle(datafolder+outputfile)